# Example 3 — Vasp Solid electrolyte 3D example

Here, we examine a small example of calculating the **fluoride ion density** confined within the solid electrolyte *BaSnF₄*, taken from a [previous study](https://pubs.acs.org/doi/full/10.1021/jacs.3c08232).  

This example demonstrates the **VASP trajectory state** and the **Gaussian cube file export function**, which together form the primary **materials science use case** of *RevelsMD*.  
Currently, this method supports **cubic** and **orthorhombic** simulation cells.

In [1]:
from revelsMD import trajectory_states, revels_3D
from tqdm import tqdm

We begin by creating a **list of VASP run files (`vaspruns`)** to load into the trajectory state.  

In [5]:
vaspruns=[]
for i in tqdm(range(1,11)):
    vaspruns.append(f'../../../examples/example_3_BaSnF4/r{i}/vasprun.xml')

100%|██████████| 10/10 [00:00<00:00, 343795.41it/s]


The **VaspRun trajectory state** can be initialized directly from this list of files, but it is essential to ensure that **forces have been saved** in the `vasprun.xml` or `vasprun.xml.gz` outputs—these are required for reduced-variance calculations.

In [6]:
TS=trajectory_states.VaspTrajectoryState(vaspruns)

Once the **TrajectoryState** is loaded, we can proceed to calculate a **density grid**.  
The trajectory used in this example originates from [this publication](https://pubs.acs.org/doi/full/10.1021/jacs.3c08232); detailed simulation settings can be found in its *Methods* section.  

The resulting grid provides a spatial representation of the **fluoride ion density** within the solid electrolyte.


In [7]:
GS=revels_3D.Revels3D.GridState(TS,'number',600,nbins=250)
GS.make_force_grid(TS,'F')
GS.get_real_density()

100%|██████████| 14999/14999 [00:03<00:00, 4946.55it/s]


While the initial grid captures the correct structural features, the smoothest and lowest-variance density is obtained using a linear combination of conventional and force-based estimators.

To construct this combination, we must choose how to divide the trajectory frames into sections for estimating variance and covariance between frames.  
Here, we compute variance using as many sections as there are frames—effectively one frame per section.

Note: This approach has not been formally validated in the literature and is included as an advanced extension for exploration.  
It is, however, conceptually consistent with the one-frame method discussed in the [linear combination study](https://pubs.aip.org/aip/jcp/article/154/19/191101/565890).


In [8]:
NGS= GS.get_lambda(TS,14999)

100%|██████████| 14999/14999 [9:48:40<00:00,  2.35s/it]  


In [9]:
NGS.write_to_cube(TS.Vasprun.start,NGS.optimal_density,'F_density.cube')


This example illustrates how *RevelsMD* integrates with VASP outputs to compute 3D densities in solid-state systems.  
By combining force-sampling and conventional density estimators, the method yields smoother and more accurate density maps, particularly useful for studying ion conduction pathways in crystalline materials.